<a href="https://colab.research.google.com/github/sakaratsooksang/arphohub/blob/main/NDVI_by_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Install Package***

In [35]:
! pip install earthengine-api --upgrade
! pip install folium
! pip install geehydro

     |████████████████████████████████| 235kB 7.7MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.272-cp37-none-any.whl size=257987 sha256=afe5e20afdd08b4d0193f6fc7627dcfd98eee31d0c37aaaafde471e79051b961
  Stored in directory: /root/.cache/pip/wheels/17/68/07/e11c6aa546c99b6e94243d9c6e1a32978f9682dd81d08f02bb
Successfully built earthengine-api
  Found existing installation: earthengine-api 0.1.269
    Uninstalling earthengine-api-0.1.269:
      Successfully uninstalled earthengine-api-0.1.269


  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10141 sha256=0fdb654ddc0fb8d4fe3a2aafb5e61bece506c8d58e6784efd30a09372f1073f7
  Stored in directory: /root/.cache/pip/wheels/c5/07/67/5fa6e7271b46bbe0acafdc7105bbee27a39ab7132d251d822d
Successfully built geehydro


# **Import Libraly**

In [36]:
import ee
import folium
import matplotlib.pyplot as plt

Authenticate


In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=FSiT5VQsSDQTE6sKcN4UFBNPLb3LzXziHKJzbgtXNAc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhv-1TCRf0BWZLeb7n5mt7tz2YepUpTSsXZBcKS7mBdH3kNs5eQY_g

Successfully saved authorization token.


testing

In [4]:
# Load a Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

# Print image object WITHOUT call to getInfo(); prints serialized request instructions.
print(img)

# Print image object WITH call to getInfo(); prints image metadata.
print(img.getInfo())

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.load",
    "arguments": {
      "id": {
        "constantValue": "LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913"
      }
    }
  }
})
{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7961, 7301], 'crs': 'EPSG:32613', 'crs_transform': [30, 0, 270285, 0, -30, 4414815]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7961, 7301], 'crs': 'EPSG:32613', 'crs_transform': [30, 0, 270285, 0, -30, 4414815]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7961, 7301], 'crs': 'EPSG:32613', 'crs_transform': [30, 0, 270285, 0, -30, 4414815]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7961, 7301], 'crs': 'EPSG:32613', 'crs_transform': [30, 0, 27

นำเข้าภาพถ่ายดาวเทียมจาก LANDSAT07 (Multispectral Satellite) <br>
<a href=https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_SR> USGS Landsat 7 Surface Reflectance Tier 1 </a>

In [55]:
thailand = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filterMetadata('country_co', 'equals', 'TH')
#นำเข้า Thailand Boundary

**พบปัญหา** Map.addLayer ไม่สามารถทำงานได้ <br>
**แก้ไข** : พบวิธีแก้ไขโดยการติดตั้ง *libraly geehydro*

In [65]:
poi = ee.Geometry.Point([100, 13]) # point of interest
Sentinel = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
Sentinel = Sentinel.filter(ee.Filter.date('2018-01-01', '2021-01-01')).mean() #นำค่าDN Valueสามปีมาเฉลี่ยกัน
Map = folium.Map(location=[13, 100], zoom_start=6) #สร้าง Basemap
Map.setOptions('HYBRID')
ndvi = Sentinel.normalizedDifference(['B4','B3'])
ndviParams = {'min': 0, 'max': 1, 
              'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]} #sample palette for NDVI
Map.addLayer(ndvi.clip(thailand), ndviParams, 'NDVI image') #ทำการนำ NDVI มา clip ด้วย Boundary ของthailand
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

ปัญหาที่พบ : ในตอนแรกพยายามทำ Sentinel เนื่องจากว่าเคยได้ยินชื่อ Landsatมานานแล้วแต่ไม่เคยเห็นข้อมูล Sentinel แล้วจึงพยายามทำ 
เพิ่งนึกได้ว่า Sentinel บินฝั่งยุโรป... <br>
เลยเปลี่ยนกลับมาใช้ Landsat7 แล้วก็ได้ผลดังแสดง
